<a href="https://colab.research.google.com/github/Gabolguima/transfer-learning/blob/main/transfer_learning_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning / Fine-tuning

Este tutorial irá guiá-lo pelo processo de uso do aprendizado por transferência (transfer learning) para treinar um classificador de imagens preciso a partir de um número relativamente pequeno de amostras de treinamento. De modo geral, o aprendizado por transferência refere-se ao processo de aproveitar o conhecimento aprendido em um modelo para o treinamento de outro modelo.

Mais especificamente, o processo envolve pegar uma rede neural existente que foi previamente treinada com bom desempenho em um conjunto de dados maior e usá-la como base para um novo modelo, que aproveita a precisão da rede anterior para uma nova tarefa. Esse método se tornou popular nos últimos anos para melhorar o desempenho de redes neurais treinadas com pequenos conjuntos de dados; a intuição é que o novo conjunto de dados pode ser pequeno demais para alcançar um bom desempenho por si só, mas sabemos que a maioria das redes neurais treinadas para aprender características de imagens frequentemente aprende características semelhantes, especialmente nas camadas iniciais, onde essas características são mais genéricas (detectores de bordas, formas, etc.).

O aprendizado por transferência tem sido amplamente possibilitado pela disponibilização de modelos de última geração como código aberto; para os modelos de melhor desempenho em tarefas de classificação de imagens (como os do [ILSVRC](http://www.image-net.org/challenges/LSVRC/)), tornou-se prática comum não apenas publicar a arquitetura, mas também liberar os pesos treinados do modelo. Isso permite que qualquer pessoa utilize esses classificadores de imagem de alto nível para impulsionar o desempenho de seus próprios modelos específicos para determinada tarefa.

###Extração de características vs. ajuste fino
Em um extremo, o aprendizado por transferência pode envolver pegar a rede pré-treinada, congelar os pesos e usar uma de suas camadas ocultas (geralmente a última) como extratora de características, utilizando essas características como entrada para uma rede neural menor.

No outro extremo, começamos com a rede pré-treinada, mas permitimos que alguns de seus pesos (geralmente os da última camada ou das últimas camadas) sejam modificados. Outro nome para esse procedimento é “ajuste fino” (fine-tuning), porque estamos ajustando levemente os pesos da rede pré-treinada para a nova tarefa. Geralmente treinamos essa rede com uma taxa de aprendizado menor, já que esperamos que as características já sejam relativamente boas e não precisem ser muito alteradas.

Às vezes, adotamos uma abordagem intermediária: congelamos apenas as camadas iniciais/genéricas, mas ajustamos as camadas finais. Qual estratégia é melhor depende do tamanho do seu conjunto de dados, do número de classes e de quanto ele se assemelha ao conjunto de dados no qual o modelo anterior foi treinado (e, portanto, se ele pode se beneficiar dos mesmos extratores de características aprendidos). Uma discussão mais detalhada sobre como escolher a melhor estratégia pode ser encontrada em [[1]](http://cs231n.github.io/transfer-learning/) [[2]](http://sebastianruder.com/transfer-learning/).

##Procedimento
Neste guia, passaremos pelo processo de carregar um classificador de imagens de última geração, com 1000 classes, o [VGG16](https://arxiv.org/pdf/1409.1556.pdf) que [venceu o desafio ImageNet em 2014](http://www.robots.ox.ac.uk/~vgg/research/very_deep/), e usá-lo como extrator de características fixo para treinar um classificador personalizado menor sobre nossas próprias imagens. Com poucas mudanças no código, você também poderá experimentar o ajuste fino (fine-tuning).

Primeiro, carregaremos o VGG16 e removeremos sua camada final, a camada softmax de classificação para 1000 classes específicas do ImageNet, e a substituiremos por uma nova camada de classificação para as classes que estamos treinando. Em seguida, congelaremos todos os pesos da rede, exceto os novos que se conectam à nova camada de classificação, e treinaremos essa nova camada sobre nosso novo conjunto de dados.

Também compararemos esse método com o treinamento de uma pequena rede neural do zero sobre o novo conjunto de dados e, como veremos, isso melhorará significativamente nossa acurácia.

Essa estratégia é eficaz mesmo para conjuntos de imagens com apenas algumas centenas de amostras. O desempenho será naturalmente menor com menos dados (dependendo do número de classes), mas ainda impressionante considerando as limitações habituais.

In [8]:
%matplotlib inline

import os
import random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.applications import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense

## Carregar o dataset cats vs dogs

### Subtarefa:
Usar `tensorflow_datasets` para carregar o conjunto de dados Cats vs Dogs.

**Raciocínio**:
Importar a biblioteca `tensorflow_datasets` e carregar o dataset `cats_vs_dogs` com as divisões e formato especificados.

In [9]:
# Carrega o dataset "cats_vs_dogs" do TensorFlow Datasets
(train_ds, val_ds, test_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

## Pré-processar o dataset

### Subtarefa:
Redimensionar as imagens para o tamanho esperado pelo VGG16 (224x224) e normalizar os valores dos pixels. Separar os dados em conjuntos de treino, validação e teste.

**Raciocínio**:
Definir a função `preprocess_image` para redimensionar e normalizar as imagens e, em seguida, aplicá-la aos conjuntos de dados.

In [10]:
# Função para redimensionar e normalizar as imagens
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Aplicando o pré-processamento e otimizando os datasets
train_ds = train_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

## Adaptar o modelo vgg16

### Subtarefa:
Modificar o modelo VGG16 pré-treinado para ter uma camada de saída com 2 neurônios (para as classes 'gato' e 'cachorro').

**Raciocínio**:
A subtarefa é modificar o modelo VGG16 para classificação binária (gatos vs cachorros). Isso envolve carregar o modelo base VGG16, adicionar uma camada Flatten e uma camada Dense de saída com dois neurônios para as duas classes.

In [11]:
# Carrega o modelo VGG16 pré-treinado no ImageNet (sem a camada de saída original)
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adiciona nova cabeça de classificação para 2 classes
x = Flatten()(vgg_base.output)
output = Dense(2, activation='softmax')(x)

# Define o novo modelo
model_vgg = Model(inputs=vgg_base.input, outputs=output)

# Exibe o resumo da arquitetura
model_vgg.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 14,764,866 (56.32 MB)

 Non-trainable params: 0 (0.00 B)

## Congelar as camadas base do vgg16

### Subtarefa:
Manter os pesos das camadas convolucionais do VGG16 fixos.

**Raciocínio**:
Iterar pelas camadas do modelo `vgg_base` e definir `trainable` como `False`, em seguida compilar o `model_vgg`.

In [12]:
# Congela todas as camadas do VGG16 (usado como extrator de características)
for layer in vgg_base.layers:
    layer.trainable = False

##Compilar o modelo com a nova camada de classificação
###Subtarefa:
Configurar o modelo para o treinamento, definindo o otimizador, a função de perda e as métricas de avaliação.

**Raciocínio:**
Após adicionar a nova camada densa ao topo do modelo VGG16 e congelar as camadas base, precisamos compilar o modelo. Nessa etapa, utilizamos o otimizador Adam por sua eficiência em problemas de classificação, e a função de perda `sparse_categorical_crossentropy`, pois os rótulos do dataset Cats vs Dogs são fornecidos como inteiros (0 ou 1), e não como vetores one-hot. Isso evita a necessidade de pré-processamento adicional e torna o treinamento mais direto. A métrica escolhida foi acurácia, que indica a proporção de predições corretas e é apropriada para tarefas de classificação binária.

In [13]:
model_vgg.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

## Treinar a nova camada de classificação

### Subtarefa:
Treinar apenas a nova camada de saída com o dataset Cats vs Dogs.

**Raciocínio**:
Treinar a nova camada de classificação do modelo VGG16 usando os conjuntos de dados de treinamento e validação pré-processados e armazenar o histórico de treinamento.

In [14]:
history_vgg = model_vgg.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 107s 181ms/step - accuracy: 0.8554 - loss: 0.3387 - val_accuracy: 0.9304 - val_loss: 0.1769
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 103s 177ms/step - accuracy: 0.9296 - loss: 0.1874 - val_accuracy: 0.8844 - val_loss: 0.3650
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 142s 177ms/step - accuracy: 0.9288 - loss: 0.2086 - val_accuracy: 0.9149 - val_loss: 0.3039
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 142s 177ms/step - accuracy: 0.9425 - loss: 0.1594 - val_accuracy: 0.9282 - val_loss: 0.2502
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 112s 192ms/step - accuracy: 0.9501 - loss: 0.1347 - val_accuracy: 0.9261 - val_loss: 0.2709
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 103s 177ms/step - accuracy: 0.9556 - loss: 0.1345 - val_accuracy: 0.8719 - val_loss: 0.6956
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 142s 177ms/step - accuracy: 0.9585 - loss: 0.1351 - val_accuracy: 0.9304 - val_loss: 0.2975
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 142s 177ms/step - accuracy: 0.9561 -

## Avaliar o modelo

### Subtarefa:
Avaliar o desempenho do modelo treinado nos dados de teste.

**Raciocínio**:
Avaliar o desempenho do modelo treinado nos dados de teste.

In [16]:
loss, accuracy = model_vgg.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

73/73 ━━━━━━━━━━━━━━━━━━━━ 11s 156ms/step - accuracy: 0.9245 - loss: 0.3647
Test Loss: 0.3982602059841156
Test Accuracy: 0.9217540621757507
